<a href="https://colab.research.google.com/github/alirezasdg1/Amazon-recommender-system/blob/master/notebooks/NLP_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## process text data

In [61]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from google.colab import files
import numpy as np

import re
import urllib
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
df_all_15 = pd.read_csv("/content/drive/My Drive/Datasets/df_all_15.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df_all_15.drop(['Unnamed: 0','tech1', 'tech2','Unnamed: 0_x','fit','similar_item','date','details','Unnamed: 0_y','reviewTime','reviewerID','reviewerName','vote','image_y','Total_review_C','Total_review_P'], axis=1,inplace=True)

In [65]:
df_all_15.head(2)

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,If you're interested in exploring any area of ...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'}
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,"Wonderful, comprehensive atlas. It's a little...",Four Stars,1438646400,{'Format:': ' Paperback'}


In [66]:
df_all_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370260 entries, 0 to 370259
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   category        370260 non-null  object 
 1   description     370260 non-null  object 
 2   title           370252 non-null  object 
 3   also_buy        370260 non-null  object 
 4   image_x         370260 non-null  object 
 5   brand           359110 non-null  object 
 6   feature         370260 non-null  object 
 7   rank            370260 non-null  object 
 8   also_view       370260 non-null  object 
 9   main_cat        369863 non-null  object 
 10  price           305197 non-null  object 
 11  asin            370260 non-null  object 
 12  overall         370260 non-null  float64
 13  verified        370260 non-null  bool   
 14  reviewText      370160 non-null  object 
 15  summary         370177 non-null  object 
 16  unixReviewTime  370260 non-null  int64  
 17  style     

In [67]:
df_all_15 = df_all_15[df_all_15['title'].notnull()]

In [70]:
df_all_15['description'] = df_all_15['description'].replace(np.nan, ' ', regex=True)
df_all_15['brand'] = df_all_15['brand'].replace(np.nan, ' ', regex=True)
df_all_15['feature'] = df_all_15['feature'].replace(np.nan, ' ', regex=True)
df_all_15['reviewText'] = df_all_15['reviewText'].replace(np.nan, ' ', regex=True)
df_all_15['summary'] = df_all_15['summary'].replace(np.nan, ' ', regex=True)
df_all_15['style'] = df_all_15['style'].replace(np.nan, ' ', regex=True)

In [72]:
df_all_15["item_text"] = df_all_15["description"]+df_all_15["brand"]+df_all_15["feature"]+df_all_15["summary"]+df_all_15['style']

In [ ]:
df_all_15["image_x"].iloc[100]


"['https://images-na.ssl-images-amazon.com/images/I/510IoiKdv-L._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31wNURrrBSL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41kb57MKTsL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/4152ysfRboL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41A8vr5krnL._SS40_.jpg']"

In [ ]:
df_all_15["image_x"].iloc[1].split(',')[0][2:-1]


'https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[25678].split(',')[0][2:-1]


'https://images-na.ssl-images-amazon.com/images/I/712AbM5F8oL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[1].split(',')[0][2:-1]

'https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[0]

"['https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31bqMEqwU3L._SS40_.jpg']"

## preprosses text data

In [73]:
def text_process(text):
        
        text = re.sub(r'<[^>]*>', ' ', text).strip()
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = re.sub('[‘’“”…]', '', text)
        text = re.sub('\n', '', text)
        nopunc = [char for char in text if char not in string.punctuation]

        nopunc = ''.join(nopunc)
        list_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
     
        stemmer_snowball = PorterStemmer()
 
        tokens_stemsnowball = [stemmer_snowball.stem(sent) for sent in list_words]

        return ' '.join(tokens_stemsnowball)


In [74]:
df_all_15['item_text'] = df_all_15['item_text'].apply(text_process)

In [75]:
df_all_15['reviewText'] = df_all_15['reviewText'].apply(text_process)

In [76]:
df_all_15['title_nlp'] = df_all_15['title'].apply(text_process)

In [78]:
df_all_15.head(1)

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style,item_text,title_nlp
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,your interest explor area new york beyond metr...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},garmingreat explor rural part stateformat pape...,delorm new york state atla amp gazett


In [79]:
df_all_15.drop(['description','brand','feature','reviewText','summary',"style"], axis=1,inplace=True)

In [80]:
df_all_15.head()

,category,title,also_buy,image_x,rank,also_view,main_cat,price,asin,overall,verified,unixReviewTime,item_text,title_nlp
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,1470009600,garmingreat explor rural part stateformat pape...,delorm new york state atla amp gazett
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,1438646400,garminfour starsformat paperback,delorm new york state atla amp gazett
2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,True,1432771200,garminsuperformat paperback,delorm new york state atla amp gazett
3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,True,1505260800,garminnecessari grid campingformat paperback,garmin delorm atla amp gazett paper map arizona
4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,False,1470614400,garmindont explor without,garmin delorm atla amp gazett paper map arizona


In [81]:
df_all_15.drop(['unixReviewTime','main_cat'],axis=1,inplace=True)

In [82]:
df_all_15.to_csv('/content/drive/My Drive/Datasets/df_all_15_text.csv')

In [83]:
pip install awscli

     |████████████████████████████████| 3.3MB 3.5MB/s 
     |████████████████████████████████| 6.6MB 30.1MB/s 
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6
  Found existing installation: botocore 1.17.59
    Uninstalling botocore-1.17.59:
      Successfully uninstalled botocore-1.17.59


In [84]:
import boto3

In [85]:
s3 = boto3.client('s3')

In [87]:
aws configure

SyntaxError: ignored